In [ ]:
# !pip install transformers datasets evaluate

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers==4.28.0
!pip install rouge_score
!pip install datasets
!pip install evaluate
!pip install wandb
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=57f78895a632f5774a4213ddf5ad62690efef88bd27d6476229078ff19394eb8
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
from fastai.imports import *
from pathlib import Path
import os
from torch.utils.data import DataLoader
import warnings,transformers,logging,torch
from transformers import TrainingArguments,Trainer
from transformers import AutoModelForSequenceClassification,AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
import datasets
from datasets import load_dataset, Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import evaluate
import numpy as np 
from sklearn import metrics
import pandas as pd 

In [ ]:
# Chooses between GPU and CPU based on availability
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# os.environ["WANDB_DISABLED"] = "true"
DEVICE

'cuda'

In [ ]:
# train_df = pd.read_csv('/content/drive/MyDrive/Information Retrieval System/Course_Project/Experiment_data/final_train.csv', index_col=0)
test_df = pd.read_csv('/content/drive/MyDrive/Information Retrieval System/Course_Project/Experiment_data/final_test.csv', index_col=0)
# val_df = pd.read_csv('/content/drive/MyDrive/Information Retrieval System/Course_Project/Experiment_data/final_val.csv', index_col=0)

In [ ]:
train_df.head()

,content,label
id,,
1,The CDC currently reports 99031 deaths. In general the discrepancies in death counts between different sources are small and explicable. The death toll stands at roughly 100000 people today.,real
2,States reported 1121 deaths a small rise from last Tuesday. Southern states reported 640 of those deaths. https://t.co/YASGRTT4ux,real
3,Politically Correct Woman (Almost) Uses Pandemic as Excuse Not to Reuse Plastic Bag https://t.co/thF8GuNFPe #coronavirus #nashville,fake
4,#IndiaFightsCorona: We have 1524 #COVID testing laboratories in India and as on 25th August 2020 36827520 tests have been done : @ProfBhargava DG @ICMRDELHI #StaySafe #IndiaWillWin https://t.co/Yh3ZxknnhZ,real
5,Populous states can generate large case counts but if you look at the new cases per million today 9 smaller states are showing more cases per million than California or Texas: AL AR ID KS KY LA MS NV and SC. https://t.co/1pYW6cWRaS,real


In [ ]:
test_df.head()

,content,label
id,,
1,Our daily update is published. States reported 734k tests 39k new cases and 532 deaths. Current hospitalizations fell below 30k for the first time since June 22. https://t.co/wzSYMe0Sht,real
2,Alfalfa is the only cure for COVID-19.,fake
3,President Trump Asked What He Would Do If He Were To Catch The Coronavirus https://t.co/3MEWhusRZI #donaldtrump #coronavirus,fake
4,States reported 630 deaths. We are still seeing a solid national decline. Death reporting lags approximately 28 days from symptom onset according to CDC models that consider lags in symptoms time in hospital and the death reporting process. https://t.co/LBmcot3h9a,real
5,This is the sixth time a global health emergency has been declared under the International Health Regulations but it is easily the most severe-@DrTedros https://t.co/JvKC0PTett,real


In [ ]:
val_df.head()

,content,label
id,,
1,Chinese converting to Islam after realising that no muslim was affected by #Coronavirus #COVD19 in the country,fake
2,11 out of 13 people (from the Diamond Princess Cruise ship) who had intially tested negative in tests in Japan were later confirmed to be positive in the United States.,fake
3,"COVID-19 Is Caused By A Bacterium, Not Virus And Can Be Treated With Aspirin",fake
4,Mike Pence in RNC speech praises Donald Trump’s COVID-19 “seamless” partnership with governors and leaves out the president's state feuds: https://t.co/qJ6hSewtgB #RNC2020 https://t.co/OFoeRZDfyY,fake
5,6/10 Sky's @EdConwaySky explains the latest #COVID19 data and government announcement. Get more on the #coronavirus data here👇 https://t.co/jvGZlSbFjH https://t.co/PygSKXesBg,real


In [ ]:
warnings.simplefilter('ignore')
logging.disable(logging.WARNING)

In [ ]:
model_name = 'microsoft/DeBERTa-V3-Base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
sep = tokenizer.sep_token
# if tokenizer.pad_token is None:
#     tokenizer.add_special_tokens({'pad_token': '[PAD]'})

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
label_encoder = LabelEncoder()
# train_df['label'] = label_encoder.fit_transform(train_df['label'])
# val_df['label'] = label_encoder.fit_transform(val_df['label'])
test_df['label'] = label_encoder.fit_transform(test_df['label'])

print('Encoded Labels: ', label_encoder.classes_)
# print(Counter(train_df['label']))
# print(Counter(val_df['label']))
print(Counter(test_df['label']))

# train_ds = Dataset.from_pandas(train_df)
# eval_ds = Dataset.from_pandas(val_df)
test_ds = Dataset.from_pandas(test_df)

Encoded Labels:  ['fake' 'real']
Counter({1: 1120, 0: 1020})


In [ ]:
def tok_func(x): return tokenizer(x["content"], truncation=True, padding='max_length', max_length=128)

tok_ds = train_ds.map(tok_func, batched=True)
tok_eval_ds = eval_ds.map(tok_func, batched=True)
tok_test_ds = test_ds.map(tok_func, batched=True)

tok_ds.save_to_disk("/content/drive/MyDrive/Information Retrieval System/Course_Project/Experiment_data/tokenized_train_ds.pk")
tok_eval_ds.save_to_disk("/content/drive/MyDrive/Information Retrieval System/Course_Project/Experiment_data/tokenized_val_ds.pk")
tok_test_ds.save_to_disk("/content/drive/MyDrive/Information Retrieval System/Course_Project/Experiment_data/tokenized_test_ds.pk")

Map:   0%|          | 0/7016 [00:00<?, ? examples/s]

Map:   0%|          | 0/2394 [00:00<?, ? examples/s]

Map:   0%|          | 0/2140 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/7016 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2394 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2140 [00:00<?, ? examples/s]

In [ ]:
tok_ds = datasets.load_from_disk('/content/drive/MyDrive/Information Retrieval System/Course_Project/Experiment_data/tokenized_train_ds.pk')
tok_eval_ds = datasets.load_from_disk('/content/drive/MyDrive/Information Retrieval System/Course_Project/Experiment_data/tokenized_val_ds.pk')
tok_test_ds = datasets.load_from_disk('/content/drive/MyDrive/Information Retrieval System/Course_Project/Experiment_data/tokenized_test_ds.pk')

In [ ]:
dds = DatasetDict({"train":tok_ds, "test": tok_eval_ds})

def compute_metrics(eval_pred):
    accuracy_metric = evaluate.load("accuracy")
    f1_metric = evaluate.load("f1")
    recall_metric = evaluate.load("recall")
    precision_metrics = evaluate.load("precision")
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    try: 
        print(metrics.classification_report(labels, preds, target_names=label_encoder.classes_))
    except:
        pass

    results = {}
    results.update(accuracy_metric.compute(predictions=preds, references = labels))
    results.update(f1_metric.compute(predictions=preds, references = labels, average="weighted"))
    results.update(recall_metric.compute(predictions=preds, references = labels, average="weighted"))
    results.update(precision_metrics.compute(predictions=preds, references = labels, average="weighted"))
    return results

In [ ]:
lr, bs = 8e-5,16
wd, epochs = 0.01,10

In [ ]:
args = TrainingArguments('/content/drive/MyDrive/Information Retrieval System/Course_Project/saved_models/DeBERTA/topic_classifier_outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,save_strategy='epoch',
    num_train_epochs=epochs, weight_decay=wd, report_to='tensorboard', dataloader_num_workers=16)

# model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder.classes_))
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
               tokenizer=tokenizer, compute_metrics=compute_metrics)

# trainer.train()

In [ ]:
results = trainer.predict(tok_test_ds)

              precision    recall  f1-score   support

        fake       0.99      0.97      0.98      1020
        real       0.97      0.99      0.98      1120

    accuracy                           0.98      2140
   macro avg       0.98      0.98      0.98      2140
weighted avg       0.98      0.98      0.98      2140

